In [145]:
using LinearAlgebra, SparseArrays
import gmsh
gmsh.initialize()

include("FEM.jl")
using .FEM

In [146]:
# Adatok
E = 2e5
ν = 0.3
thickness = 1

problem = FEM.Problem("Lshape", type="PlaneStress", E=E, ν=ν, b=thickness)

Main.FEM.Problem("Lshape", "PlaneStress", 200000.0, 0.3, 7.85e-9, 1.0)

In [147]:
gmsh.merge("postproc-test.geo")
#gmsh.merge("postproc-test.msh")

Info    : Reading 'postproc-test.geo'...
Info    : Meshing 1D...
Info    : [  0%] Meshing curve 1 (Line)
Info    : [ 20%] Meshing curve 2 (Line)
Info    : [ 40%] Meshing curve 3 (Line)
Info    : [ 50%] Meshing curve 4 (Line)
Info    : [ 70%] Meshing curve 5 (Line)
Info    : [ 90%] Meshing curve 6 (Line)
Info    : Done meshing 1D (Wall 0.000323259s, CPU 0.000309s)
Info    : Meshing 2D...
Info    : [  0%] Meshing surface 1 (Plane, Frontal-Delaunay)
Info    : [  0%] Blossom: 4 internal 4 closed
Info    : [  0%] Blossom recombination completed (Wall 5.7854e-05s, CPU 5.7e-05s): 1 quads, 0 triangles, 0 invalid quads, 0 quads with Q < 0.1, avg Q = 1, min Q = 1
Info    : [ 50%] Meshing surface 2 (Plane, Frontal-Delaunay)
Info    : Done meshing 2D (Wall 0.000272296s, CPU 0.000257s)
Info    : 5 nodes 13 elements
Info    : Writing 'postproc-test.msh'...
Info    : Done writing 'postproc-test.msh'
Info    : Done reading 'postproc-test.geo'


In [148]:
bc1 = FEM.displacementConstraint("supp", ux=0, uy=0)
ld1 = FEM.traction("load", fx=1)

("load", 1, 0, 1)

In [149]:
#gmsh.fltk.run()

In [150]:
# Néhány láthatósági beállítás 0:kikapcs. 1:bekapcs.
gmsh.option.setNumber("Geometry.CurveLabels", 0)
gmsh.option.setNumber("Geometry.Points", 0)
gmsh.option.setNumber("Geometry.Curves", 0)
gmsh.option.setNumber("Mesh.Points", 0)
gmsh.option.setNumber("Mesh.Lines", 0)
gmsh.option.setNumber("Mesh.SurfaceEdges", 0)
gmsh.option.setNumber("Mesh.NodeLabels", 0)
gmsh.option.setNumber("Mesh.LineLabels", 0)
gmsh.option.setNumber("Mesh.SurfaceLabels", 0)

In [151]:
K = FEM.stiffnessMatrixPlaneStress(problem)

10×10 SparseMatrixCSC{Float64, Int64} with 84 stored entries:
      1.07041e5   35714.3       …    2849.0          2747.25
  35714.3             1.0175e5      -2747.25       -63288.6
 -68579.6          2747.25         -41310.5         35714.3
  -2747.25         8140.01          35714.3        -46601.5
       ⋅               ⋅                 ⋅               ⋅ 
       ⋅               ⋅        …        ⋅               ⋅ 
 -41310.5        -35714.3          -68579.6         -2747.25
 -35714.3        -46601.5            2747.25         8140.01
   2849.0         -2747.25              1.07041e5  -35714.3
   2747.25       -63288.6          -35714.3             1.0175e5

In [152]:
K, f = FEM.applyBoundaryConditions!(problem, K, [bc1], [ld1])

(sparse([1, 2, 3, 4, 5, 6, 7, 8, 3, 4  …  6, 7, 8, 3, 4, 5, 7, 8, 9, 10], [1, 2, 3, 3, 3, 3, 3, 3, 4, 4  …  7, 7, 7, 8, 8, 8, 8, 8, 9, 10], [1.0, 1.0, 255392.7553927557, 35714.28571428585, -109890.1098901101, -38461.53846153854, -35612.53561253569, -35714.28571428578, 35714.28571428585, 250101.75010175043  …  38461.53846153854, 145502.6455026456, 35714.28571428572, -35714.285714285776, -173178.67317867343, 32967.03296703303, 35714.28571428572, 211640.21164021187, 1.0, 1.0], 10, 10), [0.0; 0.0; … ; 0.0; 0.0;;])

In [153]:
#det(K)

In [154]:
q = FEM.solveDisplacement(K, f)

10×1 Matrix{Float64}:
  0.0
  0.0
  6.793054753775307e-5
  1.1946101499257489e-5
  0.00013944492552528325
  1.194610149925753e-5
  6.793054753775303e-5
 -1.1946101499257452e-5
  0.0
  0.0

In [155]:
S = FEM.solveStressPlaneStress(problem, q)

"elemTags = Vector{UInt64}[[0x000000000000000d], [0x000000000000000c]]"

([[1.4142135623730923, -6.51563805580231e-17, 0.0, -6.51563805580231e-17, -0.05357999125837115, 0.0, 0.0, 0.0, 0.0, 1.4142135623730923  …  0.0, 1.4142135623730923, -6.51563805580231e-17, 0.0, -6.51563805580231e-17, -0.05357999125837115, 0.0, 0.0, 0.0, 0.0], [1.4929790667638039, 0.09189308845582685, 0.0, 0.09189308845582685, 0.4478937200291412, 0.0, 0.0, 0.0, 0.0, 1.3354480579823866  …  0.0, 1.492979066763803, -0.09189308845582655, 0.0, -0.09189308845582655, 0.4478937200291409, 0.0, 0.0, 0.0, 0.0]], [13, 12])

In [156]:
gmsh.option.setNumber("View.IntervalsType", 3)
gmsh.option.setNumber("View.VectorType", 5)
gmsh.option.setString("View.Format", "%.6g")

In [157]:
u = FEM.showResultUvec(problem, q, name="uvec", visible=false)
ux = FEM.showResultUX(problem, q, name="ux", visible=false)
uy = FEM.showResultUY(problem, q, name="uy", visible=false)
s = FEM.showResultS(problem, S, name="σ", visible=false, smooth=false)
sx = FEM.showResultSX(problem, S, name="σx", visible=true, smooth=false)
sy = FEM.showResultSY(problem, S, name="σy", visible=false, smooth=false)
sxy = FEM.showResultSXY(problem, S, name="τxy", visible=false, smooth=false)

"numElem = [13, 12]"

2-element Vector{Int64}:
 13
 12

2-element Vector{Any}:
 [1.4142135623730923, 1.4142135623730923, 1.4142135623730923]
 [1.4929790667638039, 1.3354480579823866, 1.3354480579823858, 1.492979066763803]

6

In [158]:
#gmsh.model.setCurrent(problem.name)
#pp1 = gmsh.model.occ.addPoint(0, 0, 0)
#pp2 = gmsh.model.occ.addPoint(50, 50, 0)

#pl1 = gmsh.model.occ.addLine(pp1, pp2)


#gmsh.model.occ.synchronize()

In [159]:
#FEM.plotOnPath(problem, 7, s, 100, name="σy", visible=false);

In [160]:
#gmsh.option.setNumber("PostProcessing.Binary", 1)
#gmsh.option.setNumber("PostProcessing.SaveMesh", 1)

#gmsh.write("rect.msh")
#gmsh.write("rect.opt")
#gmsh.view.write(4, "rect.pos")

In [161]:
gmsh.fltk.run()

-------------------------------------------------------
Version       : 4.12.0
License       : GNU General Public License
Build OS      : Linux64-sdk
Build date    : 20231221
Build host    : gmsh.info
Build options : 64Bit ALGLIB[contrib] ANN[contrib] Bamg Blas[petsc] Blossom Cgns DIntegration Dlopen DomHex Eigen[contrib] Fltk Gmm[contrib] Hxt Jpeg Kbipack Lapack[petsc] LinuxJoystick MathEx[contrib] Med Mesh Metis[contrib] Mmg Mpeg Netgen ONELAB ONELABMetamodel OpenCASCADE OpenCASCADE-CAF OpenGL OpenMP OptHom PETSc Parser Plugins Png Post QuadMeshingTools QuadTri Solver TetGen/BR Voro++[contrib] WinslowUntangler Zlib
FLTK version  : 1.4.0
PETSc version : 3.14.4 (real arithmtic)
OCC version   : 7.7.2
MED version   : 4.1.0
Packaged by   : geuzaine
Web site      : https://gmsh.info
Issue tracker : https://gitlab.onelab.info/gmsh/gmsh/issues
-------------------------------------------------------


In [162]:
gmsh.finalize()